In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [2]:
price_data = pd.read_csv('stockPriceData-2.csv')
price_data_500 = price_data.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = price_data[price_data['Symbol'].isin(price_data_500['Symbol'])]
df['Date'] = pd.to_datetime(df['Date'])
master_date = df.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)
df.set_index('Date', inplace=True)

df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])
df = df.sort_values(['Symbol', 'Date'])
df['PrevClose'] = df.groupby('Symbol')['Close'].shift(1)
df['returns'] = (df['Close'] - df['PrevClose']) / df['PrevClose']

In [3]:
# Shift the Close price by 21 days to get the starting point for the annualized standard deviation calculation
df['Close_shifted'] = df.groupby('Symbol')['Close'].shift(21)

# Function to calculate log returns based on the shifted Close (21 days back)
def calculate_log_returns_shifted(df):
    df['LogReturn_shifted'] = np.log(df['Close_shifted'] / df['Close_shifted'].shift(1))
    return df

# Apply the shifted log return calculation for each symbol group
df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns_shifted)

# Function to calculate annualized standard deviation over a 252-day rolling window from the 21-day shifted close
def calculate_annualized_std_shifted(df):
    df['AnnualizedStd_Shifted'] = df['LogReturn_shifted'].rolling(window=252).std() * np.sqrt(252)
    return df

# Calculate annualized standard deviation based on the 21-day shifted close prices for each symbol group
df = df.groupby('Symbol', group_keys=False).apply(calculate_annualized_std_shifted)

# Define custom momentum calculation based on rebalancing months with custom intervals
def calculate_custom_momentum_intervals(df, label, start_month, end_month):
    shifted_df = df.copy()
    
    # Calculate close prices shifted by the specified start and end months
    shifted_df[f'{label}_start'] = shifted_df.groupby('Symbol')['Close'].shift(start_month * 21)  # T-1 month back
    shifted_df[f'{label}_end'] = shifted_df.groupby('Symbol')['Close'].shift(end_month * 21)      # T-7 or T-13 months back
    
    # Calculate the momentum ratio based on specified months and normalize by annualized standard deviation
    shifted_df[label] = (
        (shifted_df[f'{label}_start'] / shifted_df[f'{label}_end'] - 1) / shifted_df['AnnualizedStd_Shifted']
    )

    return shifted_df
    # return shifted_df.dropna()  # Keep shifted columns by not dropping them here

# Define pairs for each momentum ratio (start and end month)
momentum_intervals = {'MR6': (1, 7), 'MR12': (1, 13)}

# Calculate custom momentum ratios based on the specified intervals
for label, (start_month, end_month) in momentum_intervals.items():
    df = calculate_custom_momentum_intervals(df, label, start_month, end_month)

# Reset index for further calculations
df = df.reset_index()

# Calculate the mean and standard deviation of each momentum ratio across the universe on each date
for label in momentum_intervals.keys():
    df[f'mu_{label}'] = df.groupby('Date')[label].transform('mean')
    df[f'sigma_{label}'] = df.groupby('Date')[label].transform('std')

# Calculate Z-scores for each period
for label in momentum_intervals.keys():
    df[f'Z_{label}'] = (df[label] - df[f'mu_{label}']) / df[f'sigma_{label}']

In [4]:
# Calculate Weighted Average Z-score and Normalized Momentum Score for specified combinations
comb_labels = ['Z_MR6', 'Z_MR12'] 
comb_weights = [0.5, 0.5] 

# Calculate weighted average Z-score based on combination of momentum scores
df['WeightedAvgZ_comb'] = df[comb_labels].dot(comb_weights)
df = df[['Date','Symbol','Close','Mcap','Z_MR6','Z_MR12','WeightedAvgZ_comb']]
df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])

# Normalize Z_MR6 as NormalizedMomentumScore_6_1M
df['6-1M Momentum'] = np.where(
    df['Z_MR6'] >= 0,
    1 + df['Z_MR6'],
    (1 - df['Z_MR6']) ** -1
)

# Normalize Z_MR12 as NormalizedMomentumScore_12_1M
df['12-1M Momentum'] = np.where(
    df['Z_MR12'] >= 0,
    1 + df['Z_MR12'],
    (1 - df['Z_MR12']) ** -1
)

# Normalize WeightedAvgZ_comb as NormalizedMomentumScore_6_12_Avg
df['12-1M 6-1M Momentum'] = np.where(
    df['WeightedAvgZ_comb'] >= 0,
    1 + df['WeightedAvgZ_comb'],
    (1 - df['WeightedAvgZ_comb']) ** -1
)
df = df[['Date','Symbol','Close','Mcap','6-1M Momentum','12-1M Momentum','12-1M 6-1M Momentum']]
# Assuming df is your DataFrame and 'Date' is already datetime type
momentum_cols = ['6-1M Momentum', '12-1M Momentum', '12-1M 6-1M Momentum']

# Calculate percentile ranks for each momentum column, grouped by date
for col in momentum_cols:
    df[f"{col}_pct"] = df.groupby('Date')[col].rank(pct=True)
df = df.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))


In [5]:
df = df[['Date','Symbol','6-1M Momentum_pct','12-1M Momentum_pct','12-1M 6-1M Momentum_pct']]
df

,Date,Symbol,6-1M Momentum_pct,12-1M Momentum_pct,12-1M 6-1M Momentum_pct
4189381,1995-06-16,RELIANCE,NaN,NaN,NaN
4995565,1995-06-16,TATASTEEL,NaN,NaN,NaN
2145759,1995-06-16,HINDPETRO,NaN,NaN,NaN
1854586,1995-06-16,GRASIM,NaN,NaN,NaN
786233,1995-06-16,BPCL,NaN,NaN,NaN
...,...,...,...,...,...
3588908,2025-06-13,NSLNISP,0.183051,0.028814,0.050000
1663456,2025-06-13,GENUSPOWER,0.296610,0.622034,0.527119
5576834,2025-06-13,WESTLIFE,0.533898,0.179661,0.240678
891689,2025-06-13,CCL,0.750000,0.769492,0.777119


In [6]:
df.to_csv('Mom1.csv',index=False)

,Date,Symbol,Close,Mcap,6-1M Momentum,12-1M Momentum,12-1M 6-1M Momentum,6-1M Momentum_pct,12-1M Momentum_pct,12-1M 6-1M Momentum_pct
4189381,1995-06-16,RELIANCE,13.711366,125404.734065,NaN,NaN,NaN,NaN,NaN,NaN
4995565,1995-06-16,TATASTEEL,13.317597,80205.346611,NaN,NaN,NaN,NaN,NaN,NaN
2145759,1995-06-16,HINDPETRO,34.765432,67415.040000,NaN,NaN,NaN,NaN,NaN,NaN
1854586,1995-06-16,GRASIM,129.505061,47011.311897,NaN,NaN,NaN,NaN,NaN,NaN
786233,1995-06-16,BPCL,12.497917,44992.500000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4557759,2025-06-13,SHYAMTEL,14.940000,168.373800,0.644725,0.565449,0.602490,0.009322,0.103390,0.033051
5287057,2025-06-13,UMESLTD,5.690000,150.305965,0.866428,0.949593,0.906106,0.364407,0.572881,0.509322
1480701,2025-06-13,EUROTEXIND,14.030000,122.760606,1.326644,0.810496,1.046415,0.904237,0.441525,0.655085
227553,2025-06-13,ALPSINDUS,2.960000,115.777736,0.816550,1.251287,1.013312,0.240678,0.730508,0.612712


,Date,Symbol,Close,Mcap,6-1M Momentum,12-1M Momentum,12-1M 6-1M Momentum,6-1M Momentum_pct,12-1M Momentum_pct,12-1M 6-1M Momentum_pct
4196778,2025-06-13,RELIANCE,1427.90,1.932302e+07,1.330662,0.968282,1.148952,0.905932,0.584746,0.729661
2043402,2025-06-13,HDFCBANK,1917.60,1.469046e+07,1.370101,1.904398,1.637249,0.925424,0.909322,0.922881
5032335,2025-06-13,TCS,3445.70,1.246684e+07,0.796663,0.657185,0.720234,0.196610,0.245763,0.206780
693411,2025-06-13,BHARTIARTL,1840.40,1.121612e+07,1.500232,2.353447,1.926839,0.957627,0.959322,0.963559
2233685,2025-06-13,ICICIBANK,1416.10,1.010972e+07,1.407356,1.910763,1.659059,0.936441,0.910169,0.927966
...,...,...,...,...,...,...,...,...,...,...
4557759,2025-06-13,SHYAMTEL,14.94,1.683738e+02,0.644725,0.565449,0.602490,0.009322,0.103390,0.033051
5287057,2025-06-13,UMESLTD,5.69,1.503060e+02,0.866428,0.949593,0.906106,0.364407,0.572881,0.509322
1480701,2025-06-13,EUROTEXIND,14.03,1.227606e+02,1.326644,0.810496,1.046415,0.904237,0.441525,0.655085
227553,2025-06-13,ALPSINDUS,2.96,1.157777e+02,0.816550,1.251287,1.013312,0.240678,0.730508,0.612712


,Date,Symbol,Close,Mcap,6-1M Momentum,12-1M Momentum,12-1M 6-1M Momentum,6-1M Momentum_pct,12-1M Momentum_pct,12-1M 6-1M Momentum_pct
4189381,1995-06-16,RELIANCE,13.711366,125404.734065,NaN,NaN,NaN,NaN,NaN,NaN
4995565,1995-06-16,TATASTEEL,13.317597,80205.346611,NaN,NaN,NaN,NaN,NaN,NaN
2145759,1995-06-16,HINDPETRO,34.765432,67415.040000,NaN,NaN,NaN,NaN,NaN,NaN
1854586,1995-06-16,GRASIM,129.505061,47011.311897,NaN,NaN,NaN,NaN,NaN,NaN
786233,1995-06-16,BPCL,12.497917,44992.500000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3588908,2025-06-13,NSLNISP,39.050000,114440.158443,0.792722,0.510302,0.620906,0.183051,0.028814,0.050000
1663456,2025-06-13,GENUSPOWER,370.500000,112605.359198,0.838655,1.034519,0.926842,0.296610,0.622034,0.527119
5576834,2025-06-13,WESTLIFE,716.050000,111658.090948,0.941539,0.612007,0.741823,0.533898,0.179661,0.240678
891689,2025-06-13,CCL,833.700000,111330.230424,1.107039,1.351054,1.229047,0.750000,0.769492,0.777119
